# Master dataframe en functie om forecast 2021 2023 van verbruik te maken



In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time

# show all columns in the dataframe
pd.set_option('max_columns', None)

In [3]:
# extra imports 

#!pip install altair
import altair as alt


import warnings
warnings.filterwarnings("ignore")

# Define Predict Function 

In [65]:
# Define function that returns a low, mid or high prediction of SJV_TOTAAL for each PC6 for the years 2021-2023 on the basis
# of the electricity consumption in period 2010-2020. Default prediction type is 'mid'. 

import pystan
import fbprophet
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import sklearn
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


def predict_verbruik_prophet(df_input):

    # PROPHET PREDICTION MODEL - PRODUCES LOW, MID , HIGH FORECASTS
 
    # Copy input dataframe
    df = df_input.copy()
    
    # Create a list of unique pc4
    list_of_pc4= df['PC4'].unique()
    
    print (f"\nForecasting low, mid, high cases for 2021-2023 for {len(list_of_pc4)} pc4's")
   
    # Create dataframe for forecast 2021-2023
    df_forecast = pd.DataFrame()
    df_forecast.loc[:,'ds'] = [2021, 2022, 2023]
    
    #Create output dataframe
    df_output_low = pd.DataFrame()
    df_output_mid = pd.DataFrame()
    df_output_high = pd.DataFrame()

    
    # Loop over each pc6 and predict 2021-2023      
    for pc4 in list_of_pc4:
   
        df_predict = pd.DataFrame()
        df_predict['ds'] = df['JAAR'][df['PC4'] == pc4].copy()
        df_predict['y']  = df['SJV_TOTAAL'][df['PC4'] == pc4]
        
        # Skip this loop if there are less than 8 years in dataset for this PC4
        if len(df_predict) < 9:
           print (f"Skipping forecasting low-mid-high case for 2021-2023 with prophet model for pc4 {pc4}")
           continue
        
        print (f"Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 {pc4}")

         # We fit the model by instantiating a new Prophet object.
        # Any settings to the forecasting procedure are passed into the constructor. 
        # Then you call its fit method and pass in the historical dataframe. Fitting should take 1-5 seconds.
        
        m = Prophet(growth='linear',
                    changepoints=None,
                    n_changepoints=2, #25,
                    changepoint_range=0.4, #0.8,
                    yearly_seasonality=True, #'auto',
                    weekly_seasonality=False, #'auto',
                    daily_seasonality=False, #'auto',
                    holidays=None , #holidays,
                    seasonality_mode='additive',
                    seasonality_prior_scale=10.0,
                    holidays_prior_scale=10.0,
                    changepoint_prior_scale=0.05,
                    mcmc_samples=0,
                    interval_width=0.8,
                    uncertainty_samples=1000,
                    stan_backend=None)
        m.fit(df_predict)# Create linear regression object
 


        # Create rows of output dataframe for 2021-2023
        df_copy = pd.DataFrame(df[(df['PC4'] == pc4) & (df['JAAR'] == 2020)])       

        # Skip this loop if there is no data for 2020
        if df_copy.empty:
            continue
                       
        df_temp = pd.DataFrame(np.repeat(df_copy.values,3,axis=0))
        df_temp.columns = df_copy.columns
        df_temp['JAAR'].iloc[0] = 2021
        df_temp['JAAR'].iloc[1] = 2022
        df_temp['JAAR'].iloc[2] = 2023
        
        df_temp_low = df_temp.copy()
        df_temp_mid = df_temp.copy()
        df_temp_high = df_temp.copy()
        
        # Calculate forecast 
        forecast = m.predict(df_forecast) 
                
        df_temp_low['SJV_TOTAAL'].iloc[0] = forecast['yhat_lower'].iloc[0]
        df_temp_low['SJV_TOTAAL'].iloc[1] = forecast['yhat_lower'].iloc[1]
        df_temp_low['SJV_TOTAAL'].iloc[2] = forecast['yhat_lower'].iloc[2]
 
        df_temp_mid['SJV_TOTAAL'].iloc[0] = forecast['yhat'].iloc[0]
        df_temp_mid['SJV_TOTAAL'].iloc[1] = forecast['yhat'].iloc[1]
        df_temp_mid['SJV_TOTAAL'].iloc[2] = forecast['yhat'].iloc[2]

        df_temp_high['SJV_TOTAAL'].iloc[0] = forecast['yhat_upper'].iloc[0]
        df_temp_high['SJV_TOTAAL'].iloc[1] = forecast['yhat_upper'].iloc[1]
        df_temp_high['SJV_TOTAAL'].iloc[2] = forecast['yhat_upper'].iloc[2]
       
        df_output_low = df_output_low.append(df_temp_low)  
        df_output_mid = df_output_mid.append(df_temp_mid)  
        df_output_high = df_output_high.append(df_temp_high)  
   
    return df_output_low, df_output_mid, df_output_high  



def predict_verbruik(df_input, predict_type='mid'):

    # LINEAR REGRESSION MODEL

    
    # Copy input dataframe
    df = df_input.copy()
    
    # Create a list of unique pc4
    list_of_pc4= df['PC4'].unique()
    
    print (f"\nForecasting {predict_type} case for 2021-2023 for {len(list_of_pc4)} pc4's")
   
    # Create dataframe for forecast 2021-2023
    df_forecast = pd.DataFrame()
    df_forecast.loc[:,'ds'] = [2021, 2022, 2023]
    
    #Create output dataframe
    df_output = pd.DataFrame()

    
    # Loop over each pc6 and predict 2021-2023      
    for pc4 in list_of_pc4:
   
        df_predict = pd.DataFrame()
        df_predict['ds'] = df['JAAR'][df['PC4'] == pc4].copy()
        df_predict['y']  = df['SJV_TOTAAL'][df['PC4'] == pc4]
        
        # Skip this loop if there are less than 8 years in dataset for this PC4
        if len(df_predict) < 9:
           print (f"Skipping forecasting low-mid-high case for 2021-2023 with prophet model for pc4 {pc4}")
           continue

        print (f"Forecasting {predict_type} case for 2021-2023 with linear regression model for pc4 {pc4}")

        # Create linear regression object
        regr = linear_model.LinearRegression()

        # Split the data into training/testing sets
        df_predict_X = df_predict['ds'].to_numpy()
        df_predict_Y = df_predict['y'].to_numpy()        
        
        df_predict_X = df_predict_X.reshape(-1, 1)
        df_predict_Y = df_predict_Y.reshape(-1, 1)

        # Train the model using the training sets
        regr.fit(df_predict_X, df_predict_Y)
        
        # Create rows of output dataframe for 2021-2023
        df_copy = pd.DataFrame(df[(df['PC4'] == pc4) & (df['JAAR'] == 2020)])       

        # Skip this loop if there is no data for 2020
        if df_copy.empty:
            continue
            
            
        df_temp = pd.DataFrame(np.repeat(df_copy.values,3,axis=0))
        df_temp.columns = df_copy.columns
        df_temp['JAAR'].iloc[0] = 2021
        df_temp['JAAR'].iloc[1] = 2022
        df_temp['JAAR'].iloc[2] = 2023
        
        # Calculate forecast 
        forecast = regr.predict(df_forecast)
        
        low_decr = 0.99
        high_incr= 1.01
        
        if predict_type == "low":
           df_temp['SJV_TOTAAL'].iloc[0] = forecast[0] * low_decr
           df_temp['SJV_TOTAAL'].iloc[1] = forecast[1] * low_decr * low_decr
           df_temp['SJV_TOTAAL'].iloc[2] = forecast[2] * low_decr * low_decr * low_decr
        elif predict_type == "mid":
           df_temp['SJV_TOTAAL'].iloc[0] = forecast[0]
           df_temp['SJV_TOTAAL'].iloc[1] = forecast[1]
           df_temp['SJV_TOTAAL'].iloc[2] = forecast[2]
        elif predict_type == "high":
           df_temp['SJV_TOTAAL'].iloc[0] = forecast[0] * high_incr
           df_temp['SJV_TOTAAL'].iloc[1] = forecast[1] * high_incr * high_incr
           df_temp['SJV_TOTAAL'].iloc[2] = forecast[2] * high_incr * high_incr * high_incr
        else:
           print(f"Unexpected prediction type {predict_type}")
       
        #print(df_temp)
    
        df_output = df_output.append(df_temp)  
            
    return df_output

# Set directory

In [51]:
# variables used in script
data_processed_location = '../data/processed'

if 'processed' not in os.getcwd():
    os.chdir(data_processed_location)

# Read dataframes

In [52]:
# kleinverbruikgegevens gegevens inlezen
df = pd.read_hdf('kleinverbruikgegevens_data.h5')

#Delete 2021 data by keeping JAAR < 2021
df = df[df['JAAR'] < 2021]

# Fix problem with postcodes
df['POSTCODE_VAN'] = df['POSTCODE_VAN'].str.replace(" ","")
df['POSTCODE_TOT'] = df['POSTCODE_TOT'].str.replace(" ","")

# mapping van PC4 buurt naar RES regio
df_pc4_res = pd.read_hdf('pc4_res.h5')


# Create new dataframe and fill with aggregated data 

In [53]:
# maak nieuw dataframe met de jaren
df_verbruik = pd.DataFrame(pd.date_range(start='1/1/2010', end='31/12/2020' , freq='Y'), columns=['DATUM']).sort_values(by='DATUM', ascending=False)
df_verbruik['JAAR'] = df_verbruik['DATUM'].dt.year

# df_verbruik is per pc6, dus eerst omzetten naar pc4
df_verbruik_pc4 = df.groupby(['PC4','JAAR']
                                      , as_index=False
                                     ).agg({'SJV_TOTAAL': 'sum'
                                            ,'AANSLUITINGEN_AANTAL': 'sum'  
                                            ,'LEVERINGSRICHTING_PERC': 'mean'})
# join df met df_verbruik
df_verbruik = pd.merge(df_verbruik, df_verbruik_pc4, on='JAAR', how='left')

# Merge in RES definitie

In [54]:
# Change type of PC4 for merge
df_verbruik['PC4'] = df_verbruik['PC4'].astype('int64') 

# Merge in RES 
df_verbruik = pd.merge(df_verbruik, df_pc4_res, on=['PC4'], how='left')


# QC plot for 3 PC6's

In [66]:
df1 =df_verbruik[df_verbruik['PC4'] == 4251]
df2 =df_verbruik[df_verbruik['PC4'] == 6373]
df3 =df_verbruik[df_verbruik['PC4'] == 9998]


pc6_1 = alt.Chart(df1.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),
                     ).properties(width=250,height=250)
    
pc6_2 = alt.Chart(df2.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),
                     ).properties(width=250,height=250)

pc6_3 = alt.Chart(df3.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),
                     ).properties(width=250,height=250)


pc6_1 | pc6_2 | pc6_3

alt.HConcatChart(...)

# Call simple linear regression predictie function voor low, mid, high forecast

In [67]:
df_verbruik_input = df_verbruik[(df_verbruik['PC4'] == 4251) | (df_verbruik['PC4'] == 6373) | (df_verbruik['PC4'] == 9998)]


# predict low case scenario voor 2021-2023 per PC4 
df_verbruik_low = predict_verbruik(df_verbruik_input, 'low')

# predict mid case scenario voor 2021-2023 per PC4 
df_verbruik_mid = predict_verbruik(df_verbruik_input, 'mid')

# predict high case scenario voor 2021-2023 per PC4 
df_verbruik_high = predict_verbruik(df_verbruik_input, 'high')
                     


Forecasting low case for 2021-2023 for 3 pc4's
Forecasting low case for 2021-2023 with linear regression model for pc4 4251
Forecasting low case for 2021-2023 with linear regression model for pc4 6373
Forecasting low case for 2021-2023 with linear regression model for pc4 9998

Forecasting mid case for 2021-2023 for 3 pc4's
Forecasting mid case for 2021-2023 with linear regression model for pc4 4251
Forecasting mid case for 2021-2023 with linear regression model for pc4 6373
Forecasting mid case for 2021-2023 with linear regression model for pc4 9998

Forecasting high case for 2021-2023 for 3 pc4's
Forecasting high case for 2021-2023 with linear regression model for pc4 4251
Forecasting high case for 2021-2023 with linear regression model for pc4 6373
Forecasting high case for 2021-2023 with linear regression model for pc4 9998


# QC plot for low, mid , high prediction for 3 PC6's¶

In [68]:
df1 =df_verbruik[df_verbruik['PC4'] == 4251]
df2 =df_verbruik[df_verbruik['PC4'] == 6373]
df3 =df_verbruik[df_verbruik['PC4'] == 9998]

df1_low =df_verbruik_low[df_verbruik_low['PC4'] == 4251]
df2_low =df_verbruik_low[df_verbruik_low['PC4'] == 6373]
df3_low =df_verbruik_low[df_verbruik_low['PC4'] == 9998]

df1_mid =df_verbruik_mid[df_verbruik_mid['PC4'] == 4251]
df2_mid =df_verbruik_mid[df_verbruik_mid['PC4'] == 6373]
df3_mid =df_verbruik_mid[df_verbruik_mid['PC4'] == 9998]

df1_high =df_verbruik_high[df_verbruik_high['PC4'] == 4251]
df2_high =df_verbruik_high[df_verbruik_high['PC4'] == 6373]
df3_high =df_verbruik_high[df_verbruik_high['PC4'] == 9998]

pc6_1 = alt.Chart(df1.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),
                     ).properties(width=250,height=250)
    
pc6_2 = alt.Chart(df2.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),
                     ).properties(width=250,height=250)

pc6_3 = alt.Chart(df3.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),
                     ).properties(width=250,height=250)

pc6_1_low = alt.Chart(df1_low.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('red')
                     ).properties(width=250,height=250)
    
pc6_2_low = alt.Chart(df2_low.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('red')
                     ).properties(width=250,height=250)

pc6_3_low = alt.Chart(df3_low.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('red')
                     ).properties(width=250,height=250)

pc6_1_mid = alt.Chart(df1_mid.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('green')
                     ).properties(width=250,height=250)
    
pc6_2_mid = alt.Chart(df2_mid.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('green')
                     ).properties(width=250,height=250)

pc6_3_mid = alt.Chart(df3_mid.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('green')
                     ).properties(width=250,height=250)

pc6_1_high = alt.Chart(df1_high.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('orange')
                     ).properties(width=250,height=250)
    
pc6_2_high = alt.Chart(df2_high.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('orange')
                     ).properties(width=250,height=250)

pc6_3_high = alt.Chart(df3_high.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('orange')
                     ).properties(width=250,height=250)



pc6_1 + pc6_1_low + pc6_1_mid + pc6_1_high| pc6_2 + pc6_2_low + pc6_2_mid + pc6_2_high  | pc6_3  + pc6_3_low + pc6_3_mid + pc6_3_high

alt.HConcatChart(...)

# Call prophet predictie function voor low, mid, high forecast

In [69]:
df_verbruik_input  =df_verbruik[(df_verbruik['PC4'] == 4251) | (df_verbruik['PC4'] == 6373) | (df_verbruik['PC4'] == 9998)]


# predict low, mid and high case scenario voor 2021-2023 per PC4 in a single run  
df_verbruik_low, df_verbruik_mid, df_verbruik_high = predict_verbruik_prophet(df_verbruik_input)



Forecasting low, mid, high cases for 2021-2023 for 3 pc4's
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4251
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 6373
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 9998


In [70]:
df1 =df_verbruik[df_verbruik['PC4'] == 4251]
df2 =df_verbruik[df_verbruik['PC4'] == 6373]
df3 =df_verbruik[df_verbruik['PC4'] == 9998]

df1_low =df_verbruik_low[df_verbruik_low['PC4'] == 4251]
df2_low =df_verbruik_low[df_verbruik_low['PC4'] == 6373]
df3_low =df_verbruik_low[df_verbruik_low['PC4'] == 9998]

df1_mid =df_verbruik_mid[df_verbruik_mid['PC4'] == 4251]
df2_mid =df_verbruik_mid[df_verbruik_mid['PC4'] == 6373]
df3_mid =df_verbruik_mid[df_verbruik_mid['PC4'] == 9998]

df1_high =df_verbruik_high[df_verbruik_high['PC4'] == 4251]
df2_high =df_verbruik_high[df_verbruik_high['PC4'] == 6373]
df3_high =df_verbruik_high[df_verbruik_high['PC4'] == 9998]

pc6_1 = alt.Chart(df1.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),
                     ).properties(width=250,height=250)
    
pc6_2 = alt.Chart(df2.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),
                     ).properties(width=250,height=250)

pc6_3 = alt.Chart(df3.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),
                     ).properties(width=250,height=250)

pc6_1_low = alt.Chart(df1_low.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('red')
                     ).properties(width=250,height=250)
    
pc6_2_low = alt.Chart(df2_low.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('red')
                     ).properties(width=250,height=250)

pc6_3_low = alt.Chart(df3_low.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('red')
                     ).properties(width=250,height=250)

pc6_1_mid = alt.Chart(df1_mid.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('green')
                     ).properties(width=250,height=250)
    
pc6_2_mid = alt.Chart(df2_mid.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('green')
                     ).properties(width=250,height=250)

pc6_3_mid = alt.Chart(df3_mid.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('green')
                     ).properties(width=250,height=250)

pc6_1_high = alt.Chart(df1_high.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('orange')
                     ).properties(width=250,height=250)
    
pc6_2_high = alt.Chart(df2_high.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('orange')
                     ).properties(width=250,height=250)

pc6_3_high = alt.Chart(df3_high.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('orange')
                     ).properties(width=250,height=250)



pc6_1 + pc6_1_low + pc6_1_mid + pc6_1_high| pc6_2 + pc6_2_low + pc6_2_mid + pc6_2_high  | pc6_3  + pc6_3_low + pc6_3_mid + pc6_3_high

alt.HConcatChart(...)

In [71]:
df_verbruik_input  = df_verbruik

# predict low case scenario voor 2021-2023 per PC6 
df_verbruik_low_all = predict_verbruik(df_verbruik_input, 'low')

# predict mid case scenario voor 2021-2023 per PC6 
df_verbruik_mid_all = predict_verbruik(df_verbruik_input, 'mid')

# predict high case scenario voor 2021-2023 per PC6 
df_verbruik_high_all = predict_verbruik(df_verbruik_input, 'high')
                     


Forecasting low case for 2021-2023 for 1645 pc4's
Forecasting low case for 2021-2023 with linear regression model for pc4 4251
Forecasting low case for 2021-2023 with linear regression model for pc4 4254
Forecasting low case for 2021-2023 with linear regression model for pc4 4255
Forecasting low case for 2021-2023 with linear regression model for pc4 4261
Forecasting low case for 2021-2023 with linear regression model for pc4 4264
Forecasting low case for 2021-2023 with linear regression model for pc4 4265
Forecasting low case for 2021-2023 with linear regression model for pc4 4266
Forecasting low case for 2021-2023 with linear regression model for pc4 4267
Forecasting low case for 2021-2023 with linear regression model for pc4 4268
Forecasting low case for 2021-2023 with linear regression model for pc4 4269
Forecasting low case for 2021-2023 with linear regression model for pc4 4271
Forecasting low case for 2021-2023 with linear regression model for pc4 4273
Forecasting low case for 

Forecasting low case for 2021-2023 with linear regression model for pc4 4844
Forecasting low case for 2021-2023 with linear regression model for pc4 4845
Forecasting low case for 2021-2023 with linear regression model for pc4 4847
Forecasting low case for 2021-2023 with linear regression model for pc4 4849
Forecasting low case for 2021-2023 with linear regression model for pc4 4851
Forecasting low case for 2021-2023 with linear regression model for pc4 4854
Forecasting low case for 2021-2023 with linear regression model for pc4 4855
Forecasting low case for 2021-2023 with linear regression model for pc4 4856
Forecasting low case for 2021-2023 with linear regression model for pc4 4858
Skipping forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4859
Forecasting low case for 2021-2023 with linear regression model for pc4 4861
Forecasting low case for 2021-2023 with linear regression model for pc4 4871
Forecasting low case for 2021-2023 with linear regression model for 

Forecasting low case for 2021-2023 with linear regression model for pc4 5144
Forecasting low case for 2021-2023 with linear regression model for pc4 5145
Forecasting low case for 2021-2023 with linear regression model for pc4 5146
Forecasting low case for 2021-2023 with linear regression model for pc4 5151
Forecasting low case for 2021-2023 with linear regression model for pc4 5152
Forecasting low case for 2021-2023 with linear regression model for pc4 5154
Skipping forecasting low-mid-high case for 2021-2023 with prophet model for pc4 5156
Skipping forecasting low-mid-high case for 2021-2023 with prophet model for pc4 5157
Skipping forecasting low-mid-high case for 2021-2023 with prophet model for pc4 5158
Forecasting low case for 2021-2023 with linear regression model for pc4 5161
Forecasting low case for 2021-2023 with linear regression model for pc4 5165
Forecasting low case for 2021-2023 with linear regression model for pc4 5171
Forecasting low case for 2021-2023 with linear regre

Forecasting low case for 2021-2023 with linear regression model for pc4 5422
Forecasting low case for 2021-2023 with linear regression model for pc4 5423
Forecasting low case for 2021-2023 with linear regression model for pc4 5424
Forecasting low case for 2021-2023 with linear regression model for pc4 5425
Forecasting low case for 2021-2023 with linear regression model for pc4 5427
Forecasting low case for 2021-2023 with linear regression model for pc4 5428
Forecasting low case for 2021-2023 with linear regression model for pc4 5431
Forecasting low case for 2021-2023 with linear regression model for pc4 5432
Forecasting low case for 2021-2023 with linear regression model for pc4 5433
Forecasting low case for 2021-2023 with linear regression model for pc4 5434
Forecasting low case for 2021-2023 with linear regression model for pc4 5435
Forecasting low case for 2021-2023 with linear regression model for pc4 5437
Forecasting low case for 2021-2023 with linear regression model for pc4 5438

Forecasting low case for 2021-2023 with linear regression model for pc4 5692
Forecasting low case for 2021-2023 with linear regression model for pc4 5694
Forecasting low case for 2021-2023 with linear regression model for pc4 5701
Forecasting low case for 2021-2023 with linear regression model for pc4 5702
Forecasting low case for 2021-2023 with linear regression model for pc4 5703
Forecasting low case for 2021-2023 with linear regression model for pc4 5704
Forecasting low case for 2021-2023 with linear regression model for pc4 5705
Forecasting low case for 2021-2023 with linear regression model for pc4 5706
Forecasting low case for 2021-2023 with linear regression model for pc4 5707
Forecasting low case for 2021-2023 with linear regression model for pc4 5708
Forecasting low case for 2021-2023 with linear regression model for pc4 5709
Forecasting low case for 2021-2023 with linear regression model for pc4 5711
Forecasting low case for 2021-2023 with linear regression model for pc4 5712

...

Forecasting high case for 2021-2023 with linear regression model for pc4 8141
Forecasting high case for 2021-2023 with linear regression model for pc4 8144
Forecasting high case for 2021-2023 with linear regression model for pc4 8146
Forecasting high case for 2021-2023 with linear regression model for pc4 8147
Forecasting high case for 2021-2023 with linear regression model for pc4 8148
Forecasting high case for 2021-2023 with linear regression model for pc4 8151
Forecasting high case for 2021-2023 with linear regression model for pc4 8152
Skipping forecasting low-mid-high case for 2021-2023 with prophet model for pc4 8153
Forecasting high case for 2021-2023 with linear regression model for pc4 8154
Forecasting high case for 2021-2023 with linear regression model for pc4 8196
Forecasting high case for 2021-2023 with linear regression model for pc4 8261
Forecasting high case for 2021-2023 with linear regression model for pc4 8262
Forecasting high case for 2021-2023 with linear regressio

Forecasting high case for 2021-2023 with linear regression model for pc4 9409
Forecasting high case for 2021-2023 with linear regression model for pc4 9411
Forecasting high case for 2021-2023 with linear regression model for pc4 9412
Forecasting high case for 2021-2023 with linear regression model for pc4 9413
Forecasting high case for 2021-2023 with linear regression model for pc4 9414
Forecasting high case for 2021-2023 with linear regression model for pc4 9415
Forecasting high case for 2021-2023 with linear regression model for pc4 9416
Forecasting high case for 2021-2023 with linear regression model for pc4 9417
Forecasting high case for 2021-2023 with linear regression model for pc4 9418
Forecasting high case for 2021-2023 with linear regression model for pc4 9419
Forecasting high case for 2021-2023 with linear regression model for pc4 9421
Forecasting high case for 2021-2023 with linear regression model for pc4 9422
Forecasting high case for 2021-2023 with linear regression model

Forecasting high case for 2021-2023 with linear regression model for pc4 9607
Forecasting high case for 2021-2023 with linear regression model for pc4 9608
Skipping forecasting low-mid-high case for 2021-2023 with prophet model for pc4 9609
Forecasting high case for 2021-2023 with linear regression model for pc4 9611
Forecasting high case for 2021-2023 with linear regression model for pc4 9613
Skipping forecasting low-mid-high case for 2021-2023 with prophet model for pc4 9614
Forecasting high case for 2021-2023 with linear regression model for pc4 9615
Forecasting high case for 2021-2023 with linear regression model for pc4 9616
Forecasting high case for 2021-2023 with linear regression model for pc4 9617
Forecasting high case for 2021-2023 with linear regression model for pc4 9618
Forecasting high case for 2021-2023 with linear regression model for pc4 9619
Forecasting high case for 2021-2023 with linear regression model for pc4 9621
Skipping forecasting low-mid-high case for 2021-20

Forecasting high case for 2021-2023 with linear regression model for pc4 9781
Forecasting high case for 2021-2023 with linear regression model for pc4 9784
Forecasting high case for 2021-2023 with linear regression model for pc4 9785
Forecasting high case for 2021-2023 with linear regression model for pc4 9791
Forecasting high case for 2021-2023 with linear regression model for pc4 9792
Forecasting high case for 2021-2023 with linear regression model for pc4 9793
Forecasting high case for 2021-2023 with linear regression model for pc4 9794
Forecasting high case for 2021-2023 with linear regression model for pc4 9795
Forecasting high case for 2021-2023 with linear regression model for pc4 9796
Forecasting high case for 2021-2023 with linear regression model for pc4 9797
Forecasting high case for 2021-2023 with linear regression model for pc4 9798
Forecasting high case for 2021-2023 with linear regression model for pc4 9801
Forecasting high case for 2021-2023 with linear regression model

Forecasting high case for 2021-2023 with linear regression model for pc4 9985
Forecasting high case for 2021-2023 with linear regression model for pc4 9986
Forecasting high case for 2021-2023 with linear regression model for pc4 9987
Forecasting high case for 2021-2023 with linear regression model for pc4 9988
Forecasting high case for 2021-2023 with linear regression model for pc4 9989
Forecasting high case for 2021-2023 with linear regression model for pc4 9991
Forecasting high case for 2021-2023 with linear regression model for pc4 9992
Forecasting high case for 2021-2023 with linear regression model for pc4 9993
Forecasting high case for 2021-2023 with linear regression model for pc4 9994
Forecasting high case for 2021-2023 with linear regression model for pc4 9995
Forecasting high case for 2021-2023 with linear regression model for pc4 9996
Forecasting high case for 2021-2023 with linear regression model for pc4 9997
Forecasting high case for 2021-2023 with linear regression model

In [72]:
alt.Chart(df_verbruik_low_all.reset_index()).mark_line(clip=True).encode(
    alt.X('JAAR:T'),
    alt.Y('SJV_TOTAAL:Q'),
    alt.Color('PC4:N')).properties(
    width=400,height=400)

alt.Chart(...)

In [73]:
alt.Chart(df_verbruik_mid_all.reset_index()).mark_line(clip=True).encode(
    alt.X('JAAR:T'),
    alt.Y('SJV_TOTAAL:Q'),
    alt.Color('PC4:N')).properties(
    width=400,height=400)

alt.Chart(...)

In [74]:
alt.Chart(df_verbruik_high_all.reset_index()).mark_line(clip=True).encode(
    alt.X('JAAR:T'),
    alt.Y('SJV_TOTAAL:Q'),
    alt.Color('PC4:N')).properties(
    width=400,height=400)

alt.Chart(...)

In [ ]:
df_verbruik_input  = df_verbruik

# predict low case scenario voor 2021-2023 per PC6 
df_verbruik_low_prophet_all, df_verbruik_mid_prophet_all, df_verbruik_high_prophet_all = predict_verbruik_prophet(df_verbruik_input)



Forecasting low, mid, high cases for 2021-2023 for 1645 pc4's
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4251
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4254
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4255
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4261
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4264
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4265
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4266
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4267
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4268
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4269
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4271
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4273
Forecasting low-mid-high 

Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4847
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4849
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4851
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4854
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4855
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4856
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4858
Skipping forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4859
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4861
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4871
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4872
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4873
Forecasting low-mid-high case for 2021-2023 with prophet model for pc4 4874
For